## Monolingual Norec experiments with bert
These were originally in the Experiments2 notebook. I have not run them since I moved them here, so maybe there is an undefined variable

In [1]:
from simpletransformers.ner import NERModel, NERArgs
import torch
import pandas as pd
import pickle as pk
import os
import random
import json
import time, datetime
random.seed(64)
# import nltk
import re
# from nltk.tokenize.simple import SpaceTokenizer
from helpers import *
from keep_records import Keep_records

print(f"Cuda: {torch.cuda.is_available()}")

Cuda: True


In [2]:
datafolder = "data/norne2"
train_all = datafolder + "no_bokmaal-ud-train.bmes"
dev_all = datafolder + 'no_bokmaal-ud-dev.bmes'

with open(train_all) as rf:
  tags_all = tagsset(rf.read(), separator = " ")

with open(dev_all) as rf:
  assert tagsset(rf.read(), separator = " ") <= tags_all

tags_all = list(tags_all)
datasets = [(train_all,dev_all, tags_all)]
keeper = Keep_records()
print(tags_all)

['B-GPE_ORG', 'B-EVT', 'B-LOC', 'I-EVT', 'I-GPE_LOC', 'B-MISC', 'O', 'B-PROD', 'I-PER', 'B-ORG', 'B-DRV', 'I-MISC', 'B-PER', 'I-LOC', 'I-PROD', 'I-ORG', 'B-GPE_LOC', 'I-DRV', 'I-GPE_ORG']


In [3]:
# Run bert multilingual with the settings from previous cell

family = "bert"
transformersmodel = 'bert-base-multilingual-cased'
results = []

for train_path, dev_path, tags in datasets:
    model_args = NERArgs() # New args loading fall 2020
    model_args.train_batch_size = 32
    model_args.num_train_epochs = 8
    model_args.weight_decay = 0.001
    model_args.overwrite_output_dir = True
    model_args.silent = False
    model_args.save_steps = 200000

    model = NERModel(family,transformersmodel , labels = tags,args=model_args)

    out_d = "outputs/"+transformersmodel+"_"+keeper.ts
    model.train_model(train_path, output_dir= out_d)
    print(transformersmodel, "Done training")

    result, model_outputs, predictions = model.eval_model(dev_path)

    #Record settings and results
    result["train"] = train_path
    result["dev_test"] = dev_path
    result["training_epochs"] = model_args.num_train_epochs
    result["transformer_model"] = transformersmodel
    keeper.keep(result)

df = keeper.get_df()



'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PROD', 'I-PROD', 'I-PROD', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-

In [4]:
# Run all above
df

,eval_loss,precision,recall,f1_score,train,dev_test,training_epochs,transformer_model,timestamp
0,0.03983,0.894337,0.907583,0.900911,NCRFpp/norne2/no_bokmaal-ud-train.bmes,NCRFpp/norne2/no_bokmaal-ud-dev.bmes,8,bert-base-multilingual-cased,224450
